## Build the Inference container for fastai 

In this notebook, we will build the inference container that can be used for both Batch Transform and End point

In [1]:
#change permission on executable files 
!chmod +x code/*

Build the container locally

In [18]:
!docker build -t fastaiinf .

Sending build context to Docker daemon  46.59kB
Step 1/12 : FROM fastdotai/fastai:2021-02-11
 ---> c15a6ed2e7f0
Step 2/12 : LABEL maintainer="Raj Kadiyala"
 ---> Using cache
 ---> 7c3a3c5bba0b
Step 3/12 : WORKDIR /
 ---> Using cache
 ---> 602a22496c69
Step 4/12 : RUN pip3 install --no-cache --upgrade requests
 ---> Using cache
 ---> cb58cb65144a
Step 5/12 : ENV PYTHONDONTWRITEBYTECODE=1     PYTHONUNBUFFERED=1     LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/usr/local/lib"     PYTHONIOENCODING=UTF-8     LANG=C.UTF-8     LC_ALL=C.UTF-8
 ---> Using cache
 ---> d7a4ae39c1fd
Step 6/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 39558a7c7758
Step 7/12 : RUN pip3 install --no-cache --upgrade flask gunicorn
 ---> Using cache
 ---> fb2a396b33ac
Step 8/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> bd4bcb39451c
Step 9/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 --

In [12]:
!docker images

REPOSITORY                                                                                                 TAG          IMAGE ID       CREATED         SIZE
fastaiinf                                                                                                  latest       4f4c1134454d   9 seconds ago   7.44GB
973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-containers/script-mode-container-fastai   latest       7d9a39eae960   18 hours ago    7.44GB
973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai    latest       6ce894314e19   18 hours ago    7.54GB
fastai                                                                                                     latest       6ce894314e19   18 hours ago    7.54GB
fastdotai/fastai                                                                                           2021-02-11   c15a6ed2e7f0   2 years ago     7.43GB


Authenticate to ECR using boto3 and set names for tags and ecr repository

In [13]:
import boto3
import sagemaker
ecr_namespace = "sagemaker-inference-containers/"
prefix = "script-mode-container-fastai"
role=sagemaker.get_execution_role()
ecr_repository_name = ecr_namespace + prefix
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

tag with ecr repository

In [19]:
!docker tag fastaiinf $tag_name

Create a ECR repo, if it already exists, you will get an error and you can ignore it 

Authenticate into ECR 

In [20]:
!$(aws ecr get-login --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
!aws ecr create-repository --repository-name $ecr_repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:973994155064:repository/sagemaker-inference-containers/script-mode-container-fastai",
        "registryId": "973994155064",
        "repositoryName": "sagemaker-inference-containers/script-mode-container-fastai",
        "repositoryUri": "973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-containers/script-mode-container-fastai",
        "createdAt": 1677459262.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


Push the local image to ECR

In [21]:
!docker push $tag_name

The push refers to repository [973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-containers/script-mode-container-fastai]

830bb069: Preparing 
f36eacc4: Preparing 
43552477: Preparing 
aa25ed6c: Preparing 
03119f42: Preparing 
9d9efed5: Preparing 
5dee3f41: Preparing 
e46047de: Preparing 
ea1e71e9: Preparing 
bf18a086: Preparing 
fc49132e: Preparing 
9d9efed5: Waiting g 
5da50cc0: Preparing 
722bdc07: Preparing 
b673a1d6: Preparing 
150d2459: Preparing 
6268583e: Preparing 
cc6eae8b: Preparing 
8881187d: Preparing 
5df75b44: Preparing 
30bb069: Pushed lready exists 2kB1Alatest: digest: sha256:981eff4ab4537abb40d29d3a595a5094796465d8742f14584e2e06c2332e6101 size: 4925


In [22]:
tag_name

'973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-inference-containers/script-mode-container-fastai:latest'